In [1]:
import pickle
import numpy as np

demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'
demofile = f'{demodir}/demographics.pkl'
    
sex = [] 
age = []
race = []
fc = []

demo = pickle.load(open(demofile, 'rb'))

for sub in demo['age_at_cnb']:
    for task in ['rest', 'nback', 'emoid']:
        try:
            a = demo['age_at_cnb'][sub]
            s = demo['Sex'][sub] == 'M'
            r = demo['Race'][sub] == 'AA'
            if demo['Race'][sub] not in ['AA', 'EA']:
                continue
            p = np.load(f'{demodir}/fc/{sub}_task-{task}_fc.npy')
        except:
            continue
        fc.append(p)
        age.append(a)
        sex.append(s)
        race.append(r)

age = np.array(age)
sex = np.array(sex).astype('int')
race = np.array(race).astype('int')
fc = np.stack(fc)

print([a.shape for a in [fc, age, sex, race]])

[(3849, 34716), (3849,), (3849,), (3849,)]


In [2]:
import sys

lsd = '/home/anton/Documents/Tulane/Research/LatentSimilarity/'

if lsd not in sys.path:
    sys.path.append(lsd)
    
print(sys.path)

['/home/anton/Documents/Tulane/Research/code/DynamicDict', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/lib/python3.11/site-packages', '/home/anton/Documents/Tulane/Research/LatentSimilarity/']


In [57]:
from latsim import LatSimReg, LatSimClf
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def rmse(yhat, y):
    return np.mean((yhat-y)**2)**0.5

xtr, xt, ytr, yt = train_test_split(fc, age, train_size=0.01)

reg = LatSimReg(ld=1, wd=1e-3).fit(xtr, ytr)
yhat = reg.predict(xt)
# yhat = reg.predict_proba(xt)
print(rmse(yhat, yt))
# print(roc_auc_score(yt, yhat[:,1]))
# print(np.mean(yhat == yt))

3.239835253555496


In [58]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression

reg = Ridge(alpha=1).fit(xtr, ytr)
yhat = reg.predict(xt)
print(rmse(yhat, yt))

# xtr, xt, ytr, yt = train_test_split(fc, race, stratify=race, train_size=0.8)

# reg = LogisticRegression(C=1).fit(xtr, ytr)
# yhat = reg.predict_proba(xt)
# # print(rmse(yhat, yt))
# print(roc_auc_score(yt, yhat[:,1]))
# # print(np.mean(yhat == yt))

3.1325756443507293
